## Image -> Text Task

Goal: Set up a pipeline to Claude to identify non-text parts of the image (shape, icongraphy, etc)

In [ ]:
# Having errors? Want to see the code? Look at llm_helper functions! 
from llm_helper_functions import *

#TODO
# Include headers in the functions
# Run / Test / Find Errors
    # Edit the transcription prompt to add ? for unknown characters? 
# Include a file size check somewhere in processing images
# Better Error Handleing in the final functions

# Look into OCR
# Edit last function to take the OCR Transcription 

# Go take more pictures
# Make the API Set up thing better / include constants for those paths as well 

### Folder and API Set Up

In [ ]:
# Note there is a 5MB limit on images
# It took 5 minutes to run 38 images
INPUT_FOLDER = "../data/examples/" # TODO change to ..data/input/
OUTPUT_FOLDER = "../data/output/"
OUTPUT_FILENAME = "results.csv"

API_KEY = get_api_key("credentials.txt")
HEADERS = {
    "Content-Type": "application/json",
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01"
}
MODEL = ""

### Prompts:
Feel free to change or add more!

In [ ]:
# All of these prompts will be accompanied by an image
ICON_PROMPT = "Hi! Can you identify the iconography of this gravestone? Most of the icongraphy should be towards the top of the stone. " \
"If there is no icongoraphy, just say None. Please only return exactly what the iconography is. Do not say anything else in your answer."

SHAPE_PROMPT = "Hi! Can you identify the shape of this gravestone? Please only return exactly what the shape is. Do not say anything else in your answer."

MATERIAL_PROMPT = "Hi! Can you tell me which material this gravestone is made of? It should be one of granite, marble, or slate. " \
"Please only return exactly what the material is. Do not say anything else in your answer." 

TRANSCRIPTION_PROMPT = "Hi! Can you transcribe the text on this gravestone? Please deliminate each line of the transcription with a hyphen. " \
"Please only return the transcription. Do not say anything else in your answer."

YOUR_PROMPT_HERE = ""

# You can add your prompt variable and corresponding column here
PROMPTS = [ICON_PROMPT, SHAPE_PROMPT, MATERIAL_PROMPT, TRANSCRIPTION_PROMPT] # Dont put the info prompt in here
COLUMNS = ["Image Name", "Iconography Description", "Shape Description", "Material", "Claude Transcription"] # Don't change first/last column order

# Separate Task to translate the transcription
INFO_PROMPT = "Hi! The following is a transcription from a gravestone. Each line is separated by a newline character." \
"Can you tell me the first name, middle name, last name, date of birth, date of death, age at death, and the text of the epitaph?" \
"It is common that not all of this information will be present. For any field that is not there, say 'None'. Please only return exactly " \
"the information requested, in order separated by a comma. Do not say anything else in your answer. Here is the Transcription: "

INFO_COLUMNS = ["First Name", "Middle Name", "Last Name", "Date of Birth", "Date of Death", "Age at Death", "Epitaph Text", "Claude Transcription"]



In [ ]:
def gravestone_desc(input_folder, prompts, columns, headers, debug=False):
    """
    Uses the helper function to get all the names of the images, then calls claude with each prompt for each image.
    Puts all the information for each image in a row of a dataframe.
    
    Args:
        input_folder str: Folder path with the images
        prompts list(str): List of User-Specified Prompts for Claude
        columns list(str): Corresponding list of columns to store the results of the above prompts
        debug boolean: Debug mode. Turn on if you encounter errors and want to see the full debug message from anthropic. 
    Returns:
        df(DataFrame): Dataframe with the columns specified in columns
    """

    files = list_files_in_folder(input_folder)
    all_results = []

    for image in files:

        image_result = [image]
        for prompt in prompts:
        # Call Claude
                
            result = call_claude(prompt, headers=headers, image_path=input_folder + image, debug=debug)
            image_result.append(result['content'][0]['text'])
        # Extract Text
        all_results.append(image_result)

    # Put in a dataframe and return 
    df = pd.DataFrame(all_results, columns=columns)
    return df



def transcription_info(transcriptions, prompt, columns, headers, debug=False):
    """
    Uses the helper function to get all the names of the images, then calls claude with each prompt for each image.
    Puts all the information for each image in a row of a dataframe.
    
    Args:
        input_folder str: Folder path with the images
        prompts list(str): List of User-Specified Prompts for Claude
        columns list(str): Corresponding list of columns to store the results of the above prompts
        debug boolean: Debug mode. Turn on if you encounter errors and want to see the full debug message from anthropic. 
    Returns:
        df(DataFrame): Dataframe with the columns specified in columns. 
    """
    
    all_results = []
    
    for trans in transcriptions:

        # Call Claude
        result = call_claude(prompt + trans, headers=headers, debug=debug)

        # Split on commas: (#TODO Error Handleing)
        result = str.split((result['content'][0]['text']), ",")
        
        # Basic Error Handleing for now 
        if len(result) != len(columns):
            result = [None] * len(columns)

        result.append(trans) # Include the transcription for joining purpose later

        all_results.append(result)

    # Put in a dataframe and return 
    df = pd.DataFrame(all_results, columns=columns)
    return df


### Run the code here

In [ ]:

df_desc = gravestone_desc(INPUT_FOLDER, PROMPTS, COLUMNS, HEADERS, debug=False)
df_info = transcription_info(df_desc["Claude Transcription"], INFO_PROMPT, INFO_COLUMNS, HEADERS, debug=False)
df_all = pd.concat([df_desc, df_info])
df_all.to_csv(OUTPUT_FOLDER + OUTPUT_FILENAME)

df_all.head()

=== DEBUG INFO ===
URL: https://api.anthropic.com/v1/messages
Method: POST
Headers:
  Content-Type: application/json
  x-api-key: sk-ant-api...
  anthropic-version: 2023-06-01
Data keys: ['model', 'max_tokens', 'messages']
Model: claude-sonnet-4-20250514
Message type: <class 'list'>
Response status: 200
Response headers: {'Date': 'Thu, 17 Jul 2025 16:59:11 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'anthropic-ratelimit-input-tokens-limit': '30000', 'anthropic-ratelimit-input-tokens-remaining': '28000', 'anthropic-ratelimit-input-tokens-reset': '2025-07-17T16:59:12Z', 'anthropic-ratelimit-output-tokens-limit': '8000', 'anthropic-ratelimit-output-tokens-remaining': '8000', 'anthropic-ratelimit-output-tokens-reset': '2025-07-17T16:59:11Z', 'anthropic-ratelimit-requests-limit': '50', 'anthropic-ratelimit-requests-remaining': '49', 'anthropic-ratelimit-requests-reset': '2025-07-17T16:59:10Z', 'anthropic-ratelimit-tokens-limit': '380

,Image_Name,Iconography Description,Claude Transcription
0,_DSC0437.jpeg,None,ERECTED\n- to the Memory of\n- [partially visi...
1,_DSC0421.jpeg,None,"In Memory\nof\nSARAH THURBER\nBENSON,\nrelict ..."
2,.DS_Store,I don't see an image attached to your message....,I don't see any image attached to your message...
3,_DSC0420.jpeg,None,In Memory\nof\nFRANCES BENSON\neldest daughter...
4,_DSC0416.jpeg,None,"In Memory\n-of\n-HENRY E. BENSON,\n-youngest s..."
5,_DSC0441.jpeg,None,"ASAHEL F. PROCTOR\n-BORN\n-AUG. 24, 1848.\n-DI..."
6,_DSC0457.jpeg,Crown,SACRED\n- to the memory of\n- MRS. SARAH OLNEY...
7,_DSC0466.jpeg,Three circular rosettes,"PAULINA,-\ndaughter of-\nMr Warren & Mrs Freel..."
8,_DSC0470.jpeg,None,ELLEN H. CUNLIFF\n-\nDAUGHTER OF\n-\nJOSEPH & ...
9,_DSC0446.jpeg,None,In Memory of-\nMr Charles Easterbrooks-\nwho d...
